<a href="https://colab.research.google.com/github/rotogni/diffusion-lagr/blob/google_colab/diff_lagr_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Fri Mar 28 15:12:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   37C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

# Create the 'diff-lagr' directory first
os.makedirs('/content/drive/My Drive/diff-lagr', exist_ok=True)

# Then create the subdirectories
os.makedirs('/content/drive/My Drive/diff-lagr/model_checkpoints', exist_ok=True)
os.makedirs('/content/drive/My Drive/diff-lagr/samples', exist_ok=True)

In [5]:
%cd /content

/content


In [7]:
# Set up environment for guided diffusion project
!apt-get update
!apt-get install -y libopenmpi-dev openmpi-bin
!pip install mpi4py
!CC=mpicc CXX=mpicxx pip install --no-binary=h5py h5py

# Clone the repository (replace with the actual repo URL)
!git clone -b google_colab https://github.com/rotogni/diffusion-lagr.git
%cd diffusion-lagr

# Install the package in development mode
!pip install -e .

# Verify installation
!python -c "import guided_diffusion; print('guided_diffusion installed successfully')"



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [10]:
# Set the flags
DATA_FLAGS="--dataset_path datasets/trajectories_zero_padding_prenormalise.h5 --dataset_name train"
MODEL_FLAGS="--dims 1 --image_size 300 --in_channels 3 --num_channels 32 --num_res_blocks 2 --attention_resolutions 150,75 --channel_mult 1,2,4"
DIFFUSION_FLAGS="--diffusion_steps 800 --noise_schedule tanh6,1"
TRAIN_FLAGS="--lr 1e-4 --batch_size 8 --lr_anneal_steps 500000 --save_interval 10000"

In [11]:
# Training command
!python scripts/turb_train.py $DATA_FLAGS $MODEL_FLAGS $DIFFUSION_FLAGS $TRAIN_FLAGS

Logging to /tmp/openai-2025-03-28-15-23-12-003364
creating model and diffusion...
creating data loader...
training...
-------------------------
| grad_norm  | 0.803    |
| loss       | 0.994    |
| loss_q0    | 0.967    |
| loss_q1    | 0.959    |
| loss_q2    | 0.986    |
| loss_q3    | 1.06     |
| mse        | 0.994    |
| mse_q0     | 0.967    |
| mse_q1     | 0.959    |
| mse_q2     | 0.986    |
| mse_q3     | 1.06     |
| param_norm | 86.1     |
| samples    | 8        |
| step       | 0        |
-------------------------
saving model 0...
saving model 0.9999...
-------------------------
| grad_norm  | 0.821    |
| loss       | 0.986    |
| loss_q0    | 0.984    |
| loss_q1    | 0.971    |
| loss_q2    | 0.986    |
| loss_q3    | 1        |
| mse        | 0.986    |
| mse_q0     | 0.984    |
| mse_q1     | 0.971    |
| mse_q2     | 0.986    |
| mse_q3     | 1        |
| param_norm | 86.1     |
| samples    | 88       |
| step       | 10       |
-------------------------
---------

In [14]:
# Set sampling flags
SAMPLE_FLAGS="--num_samples 16 --batch_size 8 --model_path /content/drive/MyDrive/diff-lagr/model_checkpoints/ema_0.9999_000000.pt"

#Please note that the $MODEL_FLAGS and $DIFFUSION_FLAGS should be the same as those used in training.

In [15]:
# Sampling command
!python scripts/turb_sample.py $SAMPLE_FLAGS $MODEL_FLAGS $DIFFUSION_FLAGS


Logging to /tmp/openai-2025-03-28-15-25-34-177415
creating model and diffusion...
sampling...
created 8 samples
created 16 samples
saving to /content/drive/MyDrive/diff-lagr/samples/samples_16x300x3.npz
sampling complete
[rank0]:[W328 15:26:10.080216368 ProcessGroupNCCL.cpp:1496] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [ ]:
# After sampling with the above command, it will generate a file named samples_179200x2000x3.npz (for DM-3c as an example).
# You can use the following code to read and retrieve the generated velocities:

import h5py
import numpy as np

with h5py.File('datasets/trajectories.h5', 'r') as h5f:
    rx0 = np.array(h5f.get('min'))
    rx1 = np.array(h5f.get('max'))

u3c = (np.load('/content/drive/MyDrive/diff-lagr/samples/samples_16x300x3.npz')['arr_0']+1)*(rx1-rx0)/2 + rx0
# save u3c as a file
np.save('/content/drive/MyDrive/diff-lagr/samples/u3c_samples_16x300x3.npy', u3c)
